In [15]:
import cv2 as cv

img = None

#define all needed functions
import os
import glob
import shutil

def extract_characters_from_image(image): #results in /results
    img = cv.imread(image)
    if img is None:
        print(f"Error: Could not read image {image}")
        return

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    #cv.imshow('gray image', gray)

    # Optional: improve contrast
    gray = cv.equalizeHist(gray)

    # Threshold (adaptive) – keep text white, background black
    thresh = cv.adaptiveThreshold(
        gray, 255,
        cv.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv.THRESH_BINARY_INV,
        35, 10
    )
    cv.imshow('thresholded image', thresh)

    # Morphology to clean noise (larger kernel)
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3)) #bigger numbers add background blur
    morph = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=1)  # remove specks
    morph = cv.morphologyEx(morph, cv.MORPH_CLOSE, kernel, iterations=1)  # connect strokes
    cv.imshow('morphological image', morph)

    # Use connected components instead of contours
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(morph, connectivity=8)

    # Visualization
    vis = cv.cvtColor(morph, cv.COLOR_GRAY2BGR)

    os.makedirs("results_training", exist_ok=True)

    for i in range(1, num_labels):  # skip background
        x, y, w, h, area = stats[i]

        # Filter blobs (remove tiny specks and very large areas)
        if area < 50 or w*h > 0.15*img.shape[0]*img.shape[1]:
            continue

        char_img = morph[y:y+h + 20, x:x+w]
        cv.imwrite(f'results_training/char_{i}.png', char_img)

        # Draw boxes for visualization
        cv.rectangle(vis, (x, y), (x+w, y+h + 10), (0, 255, 0), 1)

    cv.imshow("characters", vis)

    cv.waitKey(0)
    cv.destroyAllWindows()

def clear_folder(folder_path: str):
    """
    Clears all contents of the given folder but keeps the folder itself.
    If the folder does not exist, it will be created.
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
        return

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)      # delete file or symlink
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # delete folder
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

    print(f"Cleared contents of: {folder_path}")
        


In [ ]:
extract_characters_from_image('./data/tests/test5.png')

In [11]:
def run_human_trainer():
    """
    For each image in results_training, display the image and wait for the user
    to press the key corresponding to the character they see.
    The image is then renamed to char_[user_input].png and moved to results_paired_human.
    """
    import cv2 as cv
    import os
    import shutil

    src_folder = "./results_training"
    dst_folder = "./results_paired_human"
    os.makedirs(dst_folder, exist_ok=True)

    for filename in os.listdir(src_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(src_folder, filename)
            print(f"Processing: {img_path}")
            
            img = cv.imread(img_path)
            if img is None:
                print(f"Could not open {img_path}")
                continue

            # Show the image
            cv.imshow("Label this character (press key)", img)
            key = cv.waitKey(0)  # wait until a key is pressed
            cv.destroyAllWindows()

            if key == 27:  # ESC = quit
                print("Exiting labeling.")
                break
            elif key == 32:  # SPACE = skip
                print("Skipped.")
                continue
            else:
                char = chr(key)
                print(f"Labeled as: {char}")

                # Append the key to the original filename
                base, ext = os.path.splitext(filename)
                new_filename = f"{base}{char}{ext}"

                dst_path = os.path.join(dst_folder, new_filename)
                shutil.move(img_path, dst_path)
                print(f"Moved and renamed to {dst_path}")


                # # Save with new name
                # new_filename = f"char_{char}.png"
                # dst_path = os.path.join(dst_folder, new_filename)
                # shutil.move(img_path, dst_path)
                # print(f"Moved and renamed to {dst_path}")




In [8]:
clear_folder("./results_paired_Human")

Cleared contents of: ./results_paired_Human


In [17]:
run_human_trainer()

Processing: ./results_training\char_102.png
Labeled as: N
Moved and renamed to ./results_paired_human\char_102N.png
Processing: ./results_training\char_105.png
Labeled as: M
Moved and renamed to ./results_paired_human\char_105M.png
Processing: ./results_training\char_106.png
Labeled as: o
Moved and renamed to ./results_paired_human\char_106o.png
Processing: ./results_training\char_107.png
Labeled as: n
Moved and renamed to ./results_paired_human\char_107n.png
Processing: ./results_training\char_11.png
Labeled as: d
Moved and renamed to ./results_paired_human\char_11d.png
Processing: ./results_training\char_110.png
Labeled as: M
Moved and renamed to ./results_paired_human\char_110M.png
Processing: ./results_training\char_12.png
Labeled as: b
Moved and renamed to ./results_paired_human\char_12b.png
Processing: ./results_training\char_127.png
Labeled as: X
Moved and renamed to ./results_paired_human\char_127X.png
Processing: ./results_training\char_13.png
Labeled as: e
Moved and renamed t